In [1]:
!apt -qq update >/dev/null
!apt -qq install -y tesseract-ocr >/dev/null
!pip -q install --upgrade gradio sentence-transformers faiss-cpu pymupdf pytesseract transformers accelerate




W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 15.3 MB/s eta 0:00:00


In [2]:
import os, re, uuid, math, json, gc, time
import numpy as np
import gradio as gr
import fitz
import pytesseract
from PIL import Image
from io import BytesIO
from typing import List, Dict, Tuple
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TextStreamer

In [3]:
EMB_NAME = "sentence-transformers/all-MiniLM-L6-v2"
GEN_NAME = "google/flan-t5-small"

embedder = SentenceTransformer(EMB_NAME)
tok = AutoTokenizer.from_pretrained(GEN_NAME)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(GEN_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
def ocr_page_to_text(page) -> str:
    """Rasterize a page with PyMuPDF and OCR via Tesseract."""
    pix = page.get_pixmap(dpi=220)
    img = Image.open(BytesIO(pix.tobytes("png")))
    txt = pytesseract.image_to_string(img)
    return txt

def extract_pdf_text(pdf_path: str) -> Tuple[str, Dict]:
    """
    Try digital extraction first. If a page is likely scanned (too little text),
    fall back to OCR for that page. Returns full text and a per-page metadata map.
    """
    doc = fitz.open(pdf_path)
    pages_text = []
    page_meta = {}
    for i in range(len(doc)):
        page = doc[i]
        t = page.get_text("text") or ""
        digital_chars = len(t.strip())
        if digital_chars < 60:  # likely scanned or low-text: do OCR
            t = ocr_page_to_text(page)
            dtype = "ocr"
        else:
            dtype = "digital"
        pages_text.append(t)
        page_meta[i] = {"page": i+1, "doc_type": dtype}
    return "\n".join(pages_text), page_meta

def clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def chunk_text(s: str, chunk_size=900, overlap=150) -> List[str]:
    s = clean_text(s)
    chunks, start = [], 0
    while start < len(s):
        end = min(len(s), start + chunk_size)
        cut = s.rfind(".", start, end)
        if cut != -1 and cut - start > 80:
            end = cut + 1
        piece = s[start:end].strip()
        if piece:
            chunks.append(piece)
        if end >= len(s): break
        start = max(0, end - overlap)
    return chunks

def build_faiss(files: List[gr.File]) -> Tuple[str, dict]:
    """
    Build embeddings index over uploaded PDFs.
    Returns a status string for the UI and a dict state with:
      - "index": faiss index
      - "emb_dim": dimension
      - "chunks": list[str]
      - "metas": list[dict]
      - "files": list[str]
    """
    all_chunks, all_metas = [], []
    file_list = []
    for f in (files or []):
        path = f.name
        file_list.append(os.path.basename(path))
        full_text, page_meta = extract_pdf_text(path)
        if not clean_text(full_text):
            continue
        pages = list(page_meta.keys())
        chunks = chunk_text(full_text)
        for c in chunks:
            first_pg = None
            for p in range(len(pages)):
                pass
            all_chunks.append(c)
            all_metas.append({
                "filename": os.path.basename(path),
                "doc_type": "ocr" if "ocr" in [v["doc_type"] for v in page_meta.values()] else "digital",
                "page_start": None, "page_end": None
            })
    if not all_chunks:
        return "No extractable text in the PDFs. Try other files.", None

    emb = embedder.encode(all_chunks, show_progress_bar=True, convert_to_numpy=True, normalize_embeddings=True)
    dim = emb.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(emb)

    state = {"index": index, "emb_dim": dim, "chunks": all_chunks, "metas": all_metas, "files": file_list}
    return f"Indexed {len(all_chunks)} chunks from {len(file_list)} file(s).", state

def retrieve(state: dict, query: str, k: int = 5, filters: Dict = None):
    if not state or not query.strip():
        return []
    q = embedder.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    sims, idxs = state["index"].search(q, k)
    sims, idxs = sims[0], idxs[0]
    hits = []
    for score, i in zip(sims, idxs):
        if i == -1: continue
        meta = state["metas"][i]
        if filters:
            if any(meta.get(k) != v for k, v in filters.items()):
                continue
        hits.append({"text": state["chunks"][i], "score": float(score), "meta": meta, "idx": int(i)})
    return hits

In [5]:
SYS_PROMPT = (
    "You are a helpful assistant answering ONLY from the provided document context. "
    "Cite sources as [filename pX–Y] using the given metadata when possible. "
    "If the answer is not in the context, say you cannot find it in the documents."
)

In [6]:
def build_prompt(query: str, passages: List[dict], max_chars=2200):
    cited = []
    budget = max_chars
    for h in passages:
        src = h["meta"]["filename"]
        ps = h["meta"]["page_start"]; pe = h["meta"]["page_end"]
        tag = f"[{src}{'' if ps is None else f' p{ps}–{pe or ps}'}]"
        block = f"{tag} {h['text']}\n"
        if len(block) <= budget:
            cited.append(block); budget -= len(block)
        else:
            break
    ctx = "\n".join(cited) if cited else "No relevant context."
    return f"{SYS_PROMPT}\n\nContext:\n{ctx}\n\nQuestion: {query}\nAnswer:"

def generate_answer(query: str, passages: List[dict]):
    prompt = build_prompt(query, passages)
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=2048)
    out = gen_model.generate(
        **inputs, max_new_tokens=220, do_sample=False, num_beams=4, length_penalty=0.9
    )
    ans = tok.decode(out[0], skip_special_tokens=True).strip()
    # Confidence = mean of similarity scores of used chunks
    conf = float(np.mean([p["score"] for p in passages])) if passages else 0.0
    # Sources list
    srcs = []
    for p in passages:
        meta = p["meta"]
        srcs.append({
            "file": meta["filename"],
            "pages": f"{meta['page_start'] or '?'}–{meta['page_end'] or '?'}",
            "score": round(p["score"], 3)
        })
    return ans, conf, srcs

In [9]:
def ask_fn(message, history, state, top_k, filename_filter, doc_type_filter, progress=gr.Progress()):
    if state is None:
        return "Upload PDFs and click **Build Index** first."
    f = {}
    if filename_filter and filename_filter != "Any":
        f["filename"] = filename_filter
    if doc_type_filter and doc_type_filter != "Any":
        f["doc_type"] = doc_type_filter

    progress(0.4, desc="Retrieving…")
    hits = retrieve(state, message, k=top_k, filters=f)
    progress(0.7, desc="Generating answer…")
    answer, conf, sources = generate_answer(message, hits)

    # Nice, compact sources line
    src_line = " | ".join([f"{s['file']} (sim={s['score']})" for s in sources]) if sources else "No sources"
    return f"{answer}\n\n**Sources:** {src_line}\n**Confidence:** {conf:.3f} | **Chunks used:** {len(hits)}"

def save_chat(history):
    path = f"/mnt/data/rag_chat_{uuid.uuid4().hex[:8]}.txt"
    with open(path, "w") as f:
        for u,a in history:
            f.write(f"User: {u}\nAssistant: {a}\n\n")
    return path

# ---- UI ----
with gr.Blocks(theme=gr.themes.Soft(), title="Full RAG UI") as demo:
    gr.Markdown("## 🔎 Full RAG UI (Open-Source)\nUpload PDFs (digital or scanned). We’ll OCR if needed, index with embeddings/FAISS, and answer with sources & confidence.")
    with gr.Row():
        files = gr.Files(label="Upload PDF(s)", file_types=[".pdf"])
    with gr.Row():
        build = gr.Button("🧠 Build Index", variant="primary")
        status = gr.Markdown("⬆️ Add files then build the index.")
    state = gr.State(None)

    with gr.Accordion("Retrieval Settings", open=False):
        top_k = gr.Slider(2, 10, step=1, value=5, label="Top-K chunks")
        filename_filter = gr.Dropdown(["Any"], value="Any", label="Filter by file")
        doc_type_filter = gr.Dropdown(["Any","digital","ocr"], value="Any", label="Filter by doc type")

    chat = gr.Chatbot(height=420, label="Conversation")
    msg = gr.Textbox(placeholder="Ask about the documents…", lines=2)
    with gr.Row():
        send = gr.Button("Send")
        clear = gr.ClearButton([msg, chat])
        save = gr.Button("Save Chat")
        download = gr.File(label="Download chat file")

    def _post_build(files_list):
        s, st = build_faiss(files_list)
        # populate filename filter choices
        names = ["Any"] + sorted(list({os.path.basename(f.name) for f in files_list})) if files_list else ["Any"]
        return s, st, gr.update(choices=names, value="Any")

    def _user(m, h):
        return "", (h + [[m, None]]) if m else h

    def _bot(h, st, k, fn_filt, dt_filt):
        if not h: return h
        h[-1][1] = ask_fn(h[-1][0], h, st, k, fn_filt, dt_filt)
        return h

    build.click(_post_build, [files], [status, state, filename_filter])
    send.click(_user, [msg, chat], [msg, chat]).then(_bot, [chat, state, top_k, filename_filter, doc_type_filter], [chat])
    save.click(save_chat, [chat], [download])

/tmp/ipython-input-2266530336.py:41: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot(height=420, label="Conversation")


In [10]:
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://81e2629c7c725ae6ce.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
